In [1]:
from pydataset import data
import pandas as pd
import numpy as np

#viualization imports
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr, ttest_ind
from sklearn.preprocessing import MinMaxScaler,QuantileTransformer, StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


import wrangle as w 
#remove pink warning box
import warnings
warnings.filterwarnings('ignore')

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 20 to 2982282
Data columns (total 8 columns):
 #   Column               Dtype  
---  ------               -----  
 0   propertylandusedesc  object 
 1   bedroom              int64  
 2   bathroom             int64  
 3   sqft                 int64  
 4   tax_value            int64  
 5   yearbuilt            int64  
 6   taxamount            int64  
 7   county               float64
dtypes: float64(1), int64(6), object(1)
memory usage: 147.0+ MB
(1284141, 8) (428047, 8) (428047, 8)
(1284141, 8) (428047, 8) (428047, 8)


In [2]:
df=sns.load_dataset("tips")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df.sex = df.sex.map({'Female': 1, 'Male': 0})
df.smoker = df.smoker.map({'No': 0, 'Yes': 1})
df.day = df.day.map({'Thur': 0, 'Fri': 1, 'Sat': 2, 'Sun':3})
df.time = df.time.map({'Lunch': 0, 'Dinner': 1})

In [5]:
df['price_per_person']= round(df['total_bill']/df['size'],2)
df

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,1,0,3,1,2,8.49
1,10.34,1.66,0,0,3,1,3,3.45
2,21.01,3.50,0,0,3,1,3,7.00
3,23.68,3.31,0,0,3,1,2,11.84
4,24.59,3.61,1,0,3,1,4,6.15
...,...,...,...,...,...,...,...,...
239,29.03,5.92,0,0,2,1,3,9.68
240,27.18,2.00,1,1,2,1,2,13.59
241,22.67,2.00,0,1,2,1,2,11.34
242,17.82,1.75,0,0,2,1,2,8.91


In [6]:
train_val, test = train_test_split(df, train_size=0.6, random_state=1349)
train, validate = train_test_split(train_val, train_size=0.5, random_state=1349)

In [7]:
kbest_0 = SelectKBest(f_regression, k=2)

In [8]:
kbest_0

SelectKBest(k=2, score_func=<function f_regression at 0x1230144a0>)

In [9]:
X_train, y_train = train.drop(columns='tip'), train.tip

In [10]:
kbest_0.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x1230144a0>)

In [11]:
kbest_0.scores_

array([89.62055793,  0.56861843,  0.72263628,  4.77825221,  8.33333438,
       26.46464317, 13.37543537])

In [12]:
X_train.sample(5)

,total_bill,sex,smoker,day,time,size,price_per_person
64,17.59,0,0,2,1,3,5.86
178,9.60,1,1,3,1,2,4.80
238,35.83,1,0,2,1,3,11.94
126,8.52,0,0,0,0,2,4.26
22,15.77,1,0,2,1,2,7.88


In [13]:
kbest_0.get_feature_names_out()

array(['total_bill', 'size'], dtype=object)

In [14]:
model = LinearRegression()

In [15]:
rfe = RFE(model, n_features_to_select=1)

In [16]:
rfe

RFE(estimator=LinearRegression(), n_features_to_select=1)

In [17]:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=1)

In [18]:
rfe.ranking_

array([6, 4, 7, 3, 5, 1, 2])

In [19]:
pd.DataFrame(
{
    'feature': X_train.columns.to_list(),
    'rfe_ranking': rfe.ranking_
    
})

,feature,rfe_ranking
0,total_bill,6
1,sex,4
2,smoker,7
3,day,3
4,time,5
5,size,1
6,price_per_person,2


In [20]:
rfe_1 = RFE(model, n_features_to_select=1)
# fit the rfe object
rfe_1.fit(X_train, y_train)
pd.DataFrame(
{
    'feature': X_train.columns.to_list(),
    'rfe_1_ranking': rfe_1.ranking_
    
})

,feature,rfe_1_ranking
0,total_bill,6
1,sex,4
2,smoker,7
3,day,3
4,time,5
5,size,1
6,price_per_person,2


In [21]:
seq = SequentialFeatureSelector(model, n_features_to_select=2)
seq.fit(X_train, y_train)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [22]:
seq.support_

array([ True, False, False, False, False, False,  True])

In [23]:
X_train.columns[seq.support_]

Index(['total_bill', 'price_per_person'], dtype='object')

In [24]:
def select_kbest(X,y,k):
    '''this use the kbest and give u the 2 best ranking'''
    
#     this make the model 
    kbest_0 = SelectKBest(f_regression, k=k)
#     this will take the X_train and Y_train and fit it into the model
    kbest_0.fit(X_train, y_train)
#    this well get the 2 best col 
    get_feature_names_out = kbest_0.get_feature_names_out()
    
    return get_feature_names_out

In [25]:
select_kbest(X_train,y_train,2)

array(['total_bill', 'size'], dtype=object)

In [26]:
def rfe(X,y,n):
    model = LinearRegression()
    
    rfe = RFE(model, n_features_to_select=n)
    
    # fit the rfe object
    rfe.fit(X, y)
    
    #place the rfe rankings into a dataframe for easy to read 
    df = pd.DataFrame(
    {
    'feature': X.columns.to_list(),
    'rfe_ranking': rfe.ranking_
        
    })
    
    #give us the data frame back
    return df

In [27]:
rfe(X_train,y_train,1)

,feature,rfe_ranking
0,total_bill,6
1,sex,4
2,smoker,7
3,day,3
4,time,5
5,size,1
6,price_per_person,2


In [28]:
swiss = data("swiss")
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [29]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [30]:
train_val, test = train_test_split(swiss, train_size=0.6, random_state=1349)
train, validate = train_test_split(train_val, train_size=0.5, random_state=1349)

In [31]:
X_train_swiss, y_train_swiss = train.drop(columns='Fertility'), train.Fertility

In [32]:
select_kbest(X_train,y_train,2)

array(['total_bill', 'size'], dtype=object)

In [33]:
rfe(X_train_swiss,y_train_swiss,3)

,feature,rfe_ranking
0,Agriculture,2
1,Examination,1
2,Education,1
3,Catholic,3
4,Infant.Mortality,1
